In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
with open("hp_ebs.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [4]:
with open("hp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [5]:
dfs_ebs["connections"][["e_PH"]] = dfs_ebs["connections"][["e_PH"]].round(6)
dfs_tespy["connections"][["e_PH"]] = dfs_tespy["connections"][["e_PH"]].round(6)

In [6]:
columns = ["m", "p", "T", "e_PH"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [7]:
diff_to_ebs

,m,p,T,e_PH
22,0.000000,0.000000e+00,2.444620e-09,0.000009
32,0.000079,0.000000e+00,5.074101e-09,0.000145
33,0.000079,1.459999e-10,1.154451e-05,0.000003
34,0.000079,0.000000e+00,3.992981e-05,0.000062
13,0.000002,0.000000e+00,3.415439e-10,0.000266
12,0.000002,9.975535e-10,1.134165e-05,0.000227
11,0.000002,0.000000e+00,4.416582e-14,NaN
21,0.000000,0.000000e+00,5.585294e-10,0.000009
23,0.000000,0.000000e+00,8.250183e-11,0.000003
31,0.000079,0.000000e+00,2.107637e-09,0.000126


In [8]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_PH


In [9]:
# Load the CSV files
tespy_results = pd.read_csv("hp_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("hp_components_ebsilon.csv", index_col=0)

# Step 1: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 2: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 3: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns].copy()
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,COMP,125.814542,125.819130,0.004587,0.003646
1,COND,91.493965,91.482067,-0.011897,-0.013005
2,EVA,647.197052,647.061438,-0.135615,-0.020959
3,FAN,104.880549,104.855562,-0.024986,-0.023829
4,PUMP,0.010569,0.010547,-0.000022,-0.212094
5,VAL,144.574924,144.605102,0.030178,0.020869
6,TOT,1138.332290,1138.192444,-0.139847,-0.012287
